# Week 9, Day 1: Neural Network Basics

## 🎯 Today's Objectives
- Understand the perceptron (single neuron)
- Learn PyTorch tensor operations
- Build your first neural network from scratch
- Understand forward propagation

---

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

print("✅ Libraries loaded!")
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"💻 Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

✅ Libraries loaded!
🔧 PyTorch version: 2.9.1
💻 Device: CPU


---

## Part 1: The Perceptron

The simplest neural network is a **single neuron** (perceptron):

$$y = \sigma(\mathbf{w}^T\mathbf{x} + b)$$

Where:
- $\mathbf{x}$ = input vector
- $\mathbf{w}$ = weight vector
- $b$ = bias
- $\sigma$ = activation function

In [2]:
# Manual perceptron implementation
class Perceptron:
    """
    A single neuron implementation from scratch.
    """
    def __init__(self, n_inputs):
        # Initialize weights randomly
        self.weights = np.random.randn(n_inputs) * 0.01
        self.bias = 0
        
    def sigmoid(self, x):
        """Activation function."""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def forward(self, x):
        """Compute output for input x."""
        # Linear transformation
        linear = np.dot(x, self.weights) + self.bias
        # Apply activation
        return self.sigmoid(linear)
    
    def __repr__(self):
        return f"Perceptron(weights={self.weights.round(4)}, bias={self.bias:.4f})"

# Create perceptron
neuron = Perceptron(n_inputs=3)
print("PERCEPTRON:")
print(neuron)

# Test with sample input
x = np.array([0.5, -0.2, 0.8])
output = neuron.forward(x)

print(f"\nInput: {x}")
print(f"Weights: {neuron.weights.round(4)}")
print(f"Bias: {neuron.bias}")
print(f"Linear: w·x + b = {np.dot(x, neuron.weights) + neuron.bias:.4f}")
print(f"Output (after sigmoid): {output:.4f}")

PERCEPTRON:
Perceptron(weights=[ 0.005  -0.0014  0.0065], bias=0.0000)

Input: [ 0.5 -0.2  0.8]
Weights: [ 0.005  -0.0014  0.0065]
Bias: 0
Linear: w·x + b = 0.0079
Output (after sigmoid): 0.5020


### 📊 OUTPUT EXPLANATION:

- **Weights**: Randomly initialized small values (to break symmetry)
- **Bias**: Starts at 0
- **Linear output**: The weighted sum of inputs
- **Final output**: Squashed between 0-1 by sigmoid

---

## Part 2: PyTorch Tensors

PyTorch tensors are like NumPy arrays but with:
- GPU acceleration
- Automatic differentiation (autograd)

In [3]:
# Creating tensors
print("PYTORCH TENSORS")
print("="*60)

# From list
t1 = torch.tensor([1, 2, 3, 4])
print(f"From list: {t1}")

# From numpy
np_arr = np.array([[1, 2], [3, 4]])
t2 = torch.from_numpy(np_arr)
print(f"From numpy:\n{t2}")

# Random tensors
t3 = torch.randn(3, 3)  # Standard normal distribution
print(f"\nRandom (randn):\n{t3.round()}")

# Zeros and ones
zeros = torch.zeros(2, 3)
ones = torch.ones(2, 3)
print(f"\nZeros: {zeros.shape}, Ones: {ones.shape}")

# Tensor properties
print(f"\nTensor Properties:")
print(f"  Shape: {t3.shape}")
print(f"  Dtype: {t3.dtype}")
print(f"  Device: {t3.device}")
print(f"  Requires grad: {t3.requires_grad}")

PYTORCH TENSORS
From list: tensor([1, 2, 3, 4])
From numpy:
tensor([[1, 2],
        [3, 4]])

Random (randn):
tensor([[ 0.,  0.,  0.],
        [ 0., -1., -0.],
        [ 2., -1.,  0.]])

Zeros: torch.Size([2, 3]), Ones: torch.Size([2, 3])

Tensor Properties:
  Shape: torch.Size([3, 3])
  Dtype: torch.float32
  Device: cpu
  Requires grad: False


In [4]:
# Tensor operations
print("TENSOR OPERATIONS")
print("="*60)

a = torch.tensor([[1., 2.], [3., 4.]])
b = torch.tensor([[5., 6.], [7., 8.]])

print(f"a:\n{a}")
print(f"\nb:\n{b}")

# Element-wise operations
print(f"\na + b:\n{a + b}")
print(f"\na * b (element-wise):\n{a * b}")

# Matrix multiplication
print(f"\na @ b (matrix mult):\n{a @ b}")
print(f"Same as torch.mm(a, b):\n{torch.mm(a, b)}")

# Reshaping
print(f"\na.view(4): {a.view(4)}")
print(f"a.view(1, 4): {a.view(1, 4)}")
print(f"a.T (transpose):\n{a.T}")

TENSOR OPERATIONS
a:
tensor([[1., 2.],
        [3., 4.]])

b:
tensor([[5., 6.],
        [7., 8.]])

a + b:
tensor([[ 6.,  8.],
        [10., 12.]])

a * b (element-wise):
tensor([[ 5., 12.],
        [21., 32.]])

a @ b (matrix mult):
tensor([[19., 22.],
        [43., 50.]])
Same as torch.mm(a, b):
tensor([[19., 22.],
        [43., 50.]])

a.view(4): tensor([1., 2., 3., 4.])
a.view(1, 4): tensor([[1., 2., 3., 4.]])
a.T (transpose):
tensor([[1., 3.],
        [2., 4.]])


### 📊 OUTPUT EXPLANATION:

- **Element-wise (`*`)**: Multiply corresponding elements
- **Matrix multiplication (`@` or `mm`)**: True matrix product
- **view()**: Reshape without copying (like numpy reshape)
- **T**: Transpose (swap rows and columns)

---

## Part 3: Building a Neural Network in PyTorch

In [5]:
# Simple neural network with nn.Module
class SimpleNN(nn.Module):
    """
    A simple 2-layer neural network.
    
    Architecture: Input(10) -> Hidden(64) -> ReLU -> Output(1) -> Sigmoid
    """
    def __init__(self, input_size=10, hidden_size=64, output_size=1):
        super(SimpleNN, self).__init__()
        
        # Define layers
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        
        # Activations
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        """Forward pass through the network."""
        x = self.hidden(x)    # Linear transformation
        x = self.relu(x)      # Non-linearity
        x = self.output(x)    # Second linear transformation
        x = self.sigmoid(x)   # Output activation (0-1)
        return x

# Create model
model = SimpleNN(input_size=10, hidden_size=64)

print("NEURAL NETWORK ARCHITECTURE")
print("="*60)
print(model)

# Count parameters
print("\n📊 Parameter Count:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape} = {param.numel()} params")

total = sum(p.numel() for p in model.parameters())
print(f"\n  Total: {total:,} parameters")

NEURAL NETWORK ARCHITECTURE
SimpleNN(
  (hidden): Linear(in_features=10, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

📊 Parameter Count:
  hidden.weight: torch.Size([64, 10]) = 640 params
  hidden.bias: torch.Size([64]) = 64 params
  output.weight: torch.Size([1, 64]) = 64 params
  output.bias: torch.Size([1]) = 1 params

  Total: 769 parameters


In [6]:
# Test forward pass
print("FORWARD PASS TEST")
print("="*60)

# Create sample input (batch of 5 samples, 10 features each)
x = torch.randn(5, 10)
print(f"Input shape: {x.shape}")

# Forward pass
model.eval()  # Set to evaluation mode
with torch.no_grad():  # Don't track gradients
    output = model(x)

print(f"Output shape: {output.shape}")
print(f"\nOutputs (probabilities):")
for i, prob in enumerate(output.squeeze()):
    print(f"  Sample {i+1}: {prob.item():.4f}")

print("\n📊 OUTPUT EXPLANATION:")
print("  • Values between 0-1 (sigmoid activation)")
print("  • Each output is a probability")
print("  • > 0.5 → Predict class 1, < 0.5 → Predict class 0")

FORWARD PASS TEST
Input shape: torch.Size([5, 10])
Output shape: torch.Size([5, 1])

Outputs (probabilities):
  Sample 1: 0.4485
  Sample 2: 0.4418
  Sample 3: 0.4527
  Sample 4: 0.4159
  Sample 5: 0.4290

📊 OUTPUT EXPLANATION:
  • Values between 0-1 (sigmoid activation)
  • Each output is a probability
  • > 0.5 → Predict class 1, < 0.5 → Predict class 0


---

## Part 4: Using nn.Sequential (Shortcut)

In [7]:
# Faster way to define simple networks
model_seq = nn.Sequential(
    nn.Linear(10, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

print("SEQUENTIAL MODEL")
print("="*60)
print(model_seq)

# Test
x = torch.randn(3, 10)
with torch.no_grad():
    out = model_seq(x)
print(f"\nInput: {x.shape} → Output: {out.shape}")
print(f"Outputs: {out.squeeze().tolist()}")

SEQUENTIAL MODEL
Sequential(
  (0): Linear(in_features=10, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
  (5): Sigmoid()
)

Input: torch.Size([3, 10]) → Output: torch.Size([3, 1])
Outputs: [0.45738762617111206, 0.466450572013855, 0.4609096944332123]


---

## Part 5: Financial Application - Stock Feature Processing

In [8]:
# Create synthetic stock features
np.random.seed(42)
n_samples = 1000

# Features: momentum, volatility, volume, RSI, MACD, etc.
features = {
    'momentum_5d': np.random.randn(n_samples) * 0.02,
    'momentum_20d': np.random.randn(n_samples) * 0.05,
    'volatility': np.abs(np.random.randn(n_samples)) * 0.3,
    'volume_ratio': np.random.exponential(1, n_samples),
    'rsi': np.random.uniform(20, 80, n_samples),
    'macd': np.random.randn(n_samples) * 0.01,
    'bb_position': np.random.uniform(-1, 1, n_samples),
}

df = pd.DataFrame(features)

# Normalize features (important for neural networks!)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

print("STOCK FEATURE DATA")
print("="*60)
print(df.describe().round(2))

print("\n📊 After Normalization (StandardScaler):")
print(f"Mean: {X_scaled.mean(axis=0).round(2)}")
print(f"Std:  {X_scaled.std(axis=0).round(2)}")

STOCK FEATURE DATA
       momentum_5d  momentum_20d  volatility  volume_ratio      rsi     macd  \
count      1000.00       1000.00     1000.00       1000.00  1000.00  1000.00   
mean          0.00          0.00        0.24          0.97    49.93    -0.00   
std           0.02          0.05        0.18          0.98    17.38     0.01   
min          -0.06         -0.15        0.00          0.00    20.06    -0.03   
25%          -0.01         -0.03        0.10          0.28    35.09    -0.01   
50%           0.00          0.00        0.20          0.66    49.13    -0.00   
75%           0.01          0.04        0.33          1.31    64.92     0.01   
max           0.08          0.16        1.18          6.10    79.96     0.03   

       bb_position  
count      1000.00  
mean         -0.04  
std           0.57  
min          -1.00  
25%          -0.53  
50%          -0.04  
75%           0.43  
max           1.00  

📊 After Normalization (StandardScaler):
Mean: [-0.  0.  0.  0. -0.  0.

In [9]:
# Build stock prediction model
class StockPredictionNN(nn.Module):
    """
    Neural network for stock direction prediction.
    """
    def __init__(self, n_features):
        super(StockPredictionNN, self).__init__()
        
        self.network = nn.Sequential(
            nn.Linear(n_features, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # Probability of up move
        )
        
    def forward(self, x):
        return self.network(x)
    
    def predict_direction(self, x, threshold=0.5):
        """Predict direction (1=up, 0=down)."""
        prob = self.forward(x)
        return (prob > threshold).float()

# Create model
stock_model = StockPredictionNN(n_features=7)

# Test with real data
X_tensor = torch.FloatTensor(X_scaled[:10])

with torch.no_grad():
    probs = stock_model(X_tensor)
    directions = stock_model.predict_direction(X_tensor)

print("STOCK PREDICTION MODEL")
print("="*60)
print(f"Model: {stock_model}")
print(f"\n📊 Sample Predictions:")
print(f"{'Sample':<10} {'Probability':>12} {'Direction':>10}")
print("-"*35)
for i in range(10):
    dir_str = "UP" if directions[i] == 1 else "DOWN"
    print(f"{i+1:<10} {probs[i].item():>12.4f} {dir_str:>10}")

STOCK PREDICTION MODEL
Model: StockPredictionNN(
  (network): Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

📊 Sample Predictions:
Sample      Probability  Direction
-----------------------------------
1                0.4474       DOWN
2                0.4458       DOWN
3                0.4542       DOWN
4                0.4617       DOWN
5                0.4215       DOWN
6                0.4587       DOWN
7                0.4489       DOWN
8                0.4522       DOWN
9                0.4595       DOWN
10               0.4570       DOWN


### 📊 OUTPUT EXPLANATION:

- **Probability**: Confidence in UP direction (0-1)
- **Direction**: Binary prediction based on 0.5 threshold
- ⚠️ Model is UNTRAINED! Predictions are essentially random
- Tomorrow we'll learn how to train the model

---

## 📝 Day 1 Summary

### Key Concepts Learned:

1. **Perceptron**: Single neuron with weights, bias, activation
2. **PyTorch Tensors**: GPU-accelerated arrays with autograd
3. **nn.Module**: Base class for neural networks
4. **nn.Sequential**: Quick way to stack layers
5. **Forward Pass**: Data flows through network layers

### Important Points:
- Always normalize input features (StandardScaler)
- Use `model.eval()` for inference
- Use `torch.no_grad()` when not training

### Tomorrow:
- Activation functions in depth
- Loss functions (MSE, BCE, CE)
- Introduction to backpropagation

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║              DAY 1 COMPLETE: NEURAL NETWORK BASICS               ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ Perceptron (single neuron)                                   ║
║  ✓ PyTorch tensors and operations                               ║
║  ✓ Building networks with nn.Module                             ║
║  ✓ Forward pass through network                                 ║
║  ✓ Stock feature processing example                             ║
╚══════════════════════════════════════════════════════════════════╝
""")